<a href="https://colab.research.google.com/github/nooYeaT/NLP/blob/main/%EC%96%B4%ED%85%90%EC%85%98%EB%A9%94%EC%BB%A4%EB%8B%88%EC%A6%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 양방향 LSTM과 어텐션 메커니즘

### IMDB 리뷰 데이터 전처리

In [ ]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
vocab_size=10000
(X_train, y_train), (X_test, y_test)=imdb.load_data(num_words=vocab_size)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
print(max(len(l) for l in X_train))
print(sum(map(len, X_train))/len(X_train))

2494
238.71364


In [ ]:
max_len=500
X_train=pad_sequences(X_train, maxlen=max_len)
X_test=pad_sequences(X_test, maxlen=max_len)

### 바다나우 어텐션

In [ ]:
import tensorflow as tf

In [ ]:
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = Dense(units)
    self.W2 = Dense(units)
    self.V = Dense(1)

  def call(self, values, query): # 단, key와 value는 같음
    # query shape == (batch_size, hidden size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden size)
    # score 계산을 위해 뒤에서 할 덧셈을 위해서 차원을 변경해줍니다.
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

### 양방향 LSTM + 어텐션 메커니즘

In [ ]:
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Concatenate, Dropout
from tensorflow.keras import Input, Model
from tensorflow.keras import optimizers
import os

In [ ]:
# 10,000개의 단어들을 128차원의 벡터로 임베딩
sequence_input= Input(shape=(max_len, ), dtype='int32')
embedded_sequences=Embedding(vocab_size, 128, input_length=max_len)(sequence_input)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
# 양방향 LSTM 첫번째 층
lstm=Bidirectional(LSTM(64, dropout=0.5, return_sequences=True))(embedded_sequences)

# 두번째 층
lstm, forward_h, forward_c, backward_h, backward_c=Bidirectional(LSTM(64, dropout=0.5, return_sequences=True, return_state=True))(lstm)

In [ ]:
print(lstm.shape, forward_h.shape, forward_c.shape, backward_h.shape, backward_c.shape)

(None, 500, 128) (None, 64) (None, 64) (None, 64) (None, 64)


In [ ]:
state_h = Concatenate()([forward_h, backward_h]) # 은닉 상태
state_c = Concatenate()([forward_c, backward_c]) # 셀 상태

In [ ]:
attention=BahdanauAttention(64)
context_vector, attention_weights=attention(lstm, state_h)

In [ ]:
dense1=Dense(20, activation='relu')(context_vector)
dropout=Dropout(0.5)(dense1)
output=Dense(1, activation='sigmoid')(dropout)
model=Model(inputs=sequence_input, outputs=output)

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history=model.fit(X_train, y_train, epochs=3, batch_size=256, validation_data=(X_test, y_test), verbose=1)

Epoch 1/3
98/98 ━━━━━━━━━━━━━━━━━━━━ 32s 232ms/step - accuracy: 0.5600 - loss: 0.6612 - val_accuracy: 0.8614 - val_loss: 0.3281
Epoch 2/3
98/98 ━━━━━━━━━━━━━━━━━━━━ 22s 227ms/step - accuracy: 0.8858 - loss: 0.3107 - val_accuracy: 0.8821 - val_loss: 0.2877
Epoch 3/3
98/98 ━━━━━━━━━━━━━━━━━━━━ 45s 264ms/step - accuracy: 0.9237 - loss: 0.2234 - val_accuracy: 0.8813 - val_loss: 0.2970


In [ ]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))

782/782 ━━━━━━━━━━━━━━━━━━━━ 24s 30ms/step - accuracy: 0.8808 - loss: 0.2976

 테스트 정확도: 0.8804
